gross_margin_vs_volatility_2ndQ_ASachit

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from tqdm import tqdm
import os

In [ ]:
sns.set_style("darkgrid")

# File paths
gross_margin_cache_path = "data/gross_margins_cache.csv"
price_data_path = "data/final_updated_historical_stock_prices.csv"
ticker_data_path = "data/all-defense-stocks.csv"

In [ ]:
# Load Tickers
gross_df = pd.read_csv(ticker_data_path)
tickers = gross_df['Symbol'].dropna().unique().tolist()

# Step 1: Fetch Gross Margin (from cache or Yahoo Finance)

In [ ]:
# Step 1: Fetch Gross Margin (from cache or Yahoo Finance)
if os.path.exists(gross_margin_cache_path):
    gross_margin_df = pd.read_csv(gross_margin_cache_path)
    print("Loaded Gross Margin data from cache.")
else:
    print("Fetching Gross Margin data from Yahoo Finance...")
    gross_margin_data = {'Ticker': [], 'Gross Margin': []}

    for ticker in tqdm(tickers):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            gm = info.get('grossMargins', None)
            gross_margin_data['Ticker'].append(ticker)
            gross_margin_data['Gross Margin'].append(gm)
        except Exception as e:
            print(f"Error for {ticker}: {e}")
            gross_margin_data['Ticker'].append(ticker)
            gross_margin_data['Gross Margin'].append(None)

    gross_margin_df = pd.DataFrame(gross_margin_data).dropna()
    gross_margin_df.to_csv(gross_margin_cache_path, index=False)
    print(f"Saved Gross Margin data to {gross_margin_cache_path}")

# Step 2: Load price data and compute volatility

In [ ]:
# Step 2: Load price data and compute volatility
price_df = pd.read_csv(price_data_path, index_col=0, parse_dates=True)
returns = price_df.pct_change()
volatility = returns.std()

vol_df = pd.DataFrame({'Ticker': volatility.index, 'Volatility': volatility.values})


# Step 3: Merge gross margin with volatility

In [ ]:
# Step 3: Merge gross margin with volatility
analysis_df = pd.merge(gross_margin_df, vol_df, on='Ticker')
analysis_df = analysis_df.dropna()

# Step 4: Scatter Plot - Gross Margin vs Volatility

In [ ]:
# Step 4: Scatter Plot - Gross Margin vs Volatility
plt.figure(figsize=(10, 6))
sns.scatterplot(data=analysis_df, x='Gross Margin', y='Volatility', hue='Ticker', palette='tab10')
plt.title("Gross Margin vs. Stock Volatility")
plt.xlabel("Gross Margin")
plt.ylabel("Volatility (Std. Dev of Returns)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='x-small')
plt.tight_layout()
plt.savefig("plots/gross_margin_vs_volatility.png", dpi=300)
plt.show()


# Step 5: Box Plot - Margin Tier vs Volatility

In [ ]:
# Step 5: Box Plot - Margin Tier vs Volatility
bins = [0, 0.25, 0.5, 0.75, 1]
labels = ['Low', 'Medium', 'High', 'Very High']
analysis_df['Margin Tier'] = pd.cut(analysis_df['Gross Margin'], bins=bins, labels=labels)


In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=analysis_df, x='Margin Tier', y='Volatility')
plt.title("Volatility by Gross Margin Tier")
plt.xlabel("Gross Margin Tier")
plt.ylabel("Volatility")
plt.tight_layout()
plt.savefig("plots/volatility_by_margin_tier.png", dpi=300)
plt.show()
